#### Send a request with the same location paramaters (location, radius, etc)

In [2]:
import requests
import os
import pandas as pd
import numpy as np
import json
#from pandas import json_normalize
import sqlite3
yelp_api = os.environ['yelp_api']

In [4]:
Yelp_url = 'https://api.yelp.com/v3/businesses/search?latitude=44.38&longitude=-79.70&radius=1000&categories=restaurants, All&limit=50'
yelp_headers = {
    "Accept": "application/json",
    "Authorization": "Bearer" + " " + yelp_api
}

##### """EXPLORING AND CHECKING RESQUETS DATA"""

In [5]:
yelp_res = requests.get(Yelp_url, headers=yelp_headers)
yelp_json = yelp_res.json()

In [6]:
yelp_res.status_code

200

In [7]:
type(yelp_json['businesses']), type(yelp_json['total']), type(yelp_json['region'])

(list, int, dict)

In [8]:
"""yelp_json['total'] is total res return, no need to check further, yelp_json['businesses'] is data we looking for"""
yelp_json['businesses'][0], yelp_json['region']

({'id': '0ALYlK6edo8OnbiwH-b7fA',
  'alias': 'lazy-tulip-cafe-barrie',
  'name': 'Lazy Tulip Cafe',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/hrkM-QFQP4jogKvj72phsA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/lazy-tulip-cafe-barrie?adjust_creative=R3IyOYuVXLt1MMjX1HFo3g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=R3IyOYuVXLt1MMjX1HFo3g',
  'review_count': 59,
  'categories': [{'alias': 'cafes', 'title': 'Cafes'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
   {'alias': 'sandwiches', 'title': 'Sandwiches'}],
  'rating': 4.0,
  'coordinates': {'latitude': 44.37958, 'longitude': -79.69345},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '151 Bradford Street',
   'address2': None,
   'address3': '',
   'city': 'Barrie',
   'zip_code': 'L4N 3B4',
   'country': 'CA',
   'state': 'ON',
   'display_address': ['151 Bradford Street', 'Barrie, ON L4N 3B4', 'Canada']},
  'phone': '+17052520910',
  

In [17]:
def get_popolatiry(fsq_id):
    """return popularity data for the restaurant according to unique their fsq_id"""
    poplt_res = requests.get(fsq_url_poplt+fsq_id+fsq_fields, headers=fsq_headers)
    return poplt_res.json()

def write_json(respones_data, name_json):
    """make a json file if not exit with data file and name"""
    """name_json has to have prefix .json, and in string type"""
    with open(name_json, "w") as opf:
        json.dump(respones_data, opf)

def check_empty_data(clm):
    """check nan, empty list [], dict {} column with more than 40% within column"""
    count = 0
    nan_val = None
    for value in clm:
        #null_val = ["[]", "{}" "''", "NaN", "NULL"]
        if value == [] or value == {} or value == () or value == 'NaN':
            nan_val = value
            #print(value)
            count += 1
    #("null values percent: {}%".format(count/len(clm)*100))
    if (count/len(clm)*100) > 40:
        return True
    else:
        return False
    
def drop_na(df):
    """drop all columns with nan or empty list value more than 40% within column, using function check_empty_data"""
    nan_value = np.nan
    for col in df.columns:
        #bool_val, nul_val = check_empty_column(df[col])
        if check_empty_data(df[col]) == True:
            df.drop(col, axis=1, inplace=True)
            
def high_empty_rate(df):
    """the function will return table of isnull values and True False column with columns of data, with more than 40% of emtpty rows in column"""
    bool_list = []
    col_list = []
    is_nul = [val for val in df.isnull().sum()]
    for col in df.columns:
        TF_value = check_empty_data(df[col])
        col_list.append(col)
        bool_list.append(TF_value)
    list_of_tuples = list(zip(col_list, is_nul, bool_list))
    #isnull_check = pd.DataFrame(df_fsq_revln.isnull().sum())
    return pd.DataFrame(list_of_tuples, columns=['columnName','is-nul-check','40%-or higher-empty-data'])

In [18]:
#save json file
write_json(yelp_json['businesses'], "yelp_res_data.json")

In [19]:
df_yelp = pd.read_json("yelp_res_data.json")

In [20]:
df_yelp.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0ALYlK6edo8OnbiwH-b7fA,lazy-tulip-cafe-barrie,Lazy Tulip Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/hrkM-Q...,False,https://www.yelp.com/biz/lazy-tulip-cafe-barri...,59,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.0,"{'latitude': 44.37958, 'longitude': -79.69345}",[],$$,"{'address1': '151 Bradford Street', 'address2'...",17052520910,+1 705-252-0910,531.023427
1,FEV75sLYoN1dvSpUCyIePA,pie-wood-fired-pizza-joint-barrie-4,Pie Wood Fired Pizza Joint,https://s3-media4.fl.yelpcdn.com/bphoto/-vGeWK...,False,https://www.yelp.com/biz/pie-wood-fired-pizza-...,69,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 44.3811166940956, 'longitude': -7...",[],$$,"{'address1': '11 Victoria St', 'address2': '',...",17057287952,+1 705-728-7952,748.207341
2,s-noC8RmKF6FF2TZscl4Ig,the-farmhouse-restaurant-barrie,The Farmhouse Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/02rh8p...,False,https://www.yelp.com/biz/the-farmhouse-restaur...,84,"[{'alias': 'southern', 'title': 'Southern'}, {...",3.5,"{'latitude': 44.3746199, 'longitude': -79.6896...",[],$$,"{'address1': '268 Bradford Street', 'address2'...",17057370522,+1 705-737-0522,1018.732499
3,efeu6n1otNmaJPhwNLbLLw,il-buco-ristorante-barrie,Il Buco Ristorante,https://s3-media4.fl.yelpcdn.com/bphoto/sE-WPP...,False,https://www.yelp.com/biz/il-buco-ristorante-ba...,24,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 44.38914, 'longitude': -79.68919}",[],$$,"{'address1': '31 Dunlop St E', 'address2': '',...",17057359595,+1 705-735-9595,1320.964019
4,ye_x-eM4Cl7zkjxIAHSj3g,casa-mia-restaurant-barrie,Casa Mia Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/Vf4WV0...,False,https://www.yelp.com/biz/casa-mia-restaurant-b...,23,"[{'alias': 'greek', 'title': 'Greek'}]",3.5,"{'latitude': 44.38801, 'longitude': -79.69337}",[],$$,"{'address1': '88 Dunlop Street W', 'address2':...",17057373663,+1 705-737-3663,1061.088036
5,i2lVeikx7Hj2ep9Y-zpp3Q,the-north-restaurant-barrie,The North Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/AaY3C7...,False,https://www.yelp.com/biz/the-north-restaurant-...,23,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 44.38873, 'longitude': -79.6925}",[],$$$,"{'address1': '49 Mary Street', 'address2': '',...",17058122192,+1 705-812-2192,1147.843371
6,gQEegmEvAh2btKQDcPd6Yg,andys-place-barrie,Andy's Place,https://s3-media4.fl.yelpcdn.com/bphoto/zyR8s-...,False,https://www.yelp.com/biz/andys-place-barrie?ad...,7,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 44.38115, 'longitude': -79.70649}",[],$$,"{'address1': '19 Hart Drive', 'address2': '', ...",17057219537,+1 705-721-9537,520.347698
7,OIoDkTi8qTpyyvmoXigJfg,bull-and-barrel-pub-barrie,Bull and Barrel Pub,https://s3-media4.fl.yelpcdn.com/bphoto/0AyZ5m...,False,https://www.yelp.com/biz/bull-and-barrel-pub-b...,9,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 44.3839, 'longitude': -79.71124}",[],$$,"{'address1': '75 Cedar Pointe Drive', 'address...",17057194600,+1 705-719-4600,924.857486
8,kHq6MchLviFUvR_r3R5-pg,zios-by-giancarlo-barrie,Zio's By Giancarlo,https://s3-media2.fl.yelpcdn.com/bphoto/_XVjAP...,False,https://www.yelp.com/biz/zios-by-giancarlo-bar...,6,"[{'alias': 'italian', 'title': 'Italian'}]",3.5,"{'latitude': 44.37519, 'longitude': -79.69955}",[],NaN,"{'address1': '110 Anne Street S', 'address2': ...",17052525410,+1 705-252-5410,544.163700
9,rgfmuPbGP4__ymIPw83uZA,popeyes-louisiana-kitchen-barrie,Popeyes® Louisiana Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/2kkmUd...,False,https://www.yelp.com/biz/popeyes-louisiana-kit...,12,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 44.382589, 'longitude': -79.7051327}",[],NaN,"{'address1': '7 Anne St S', 'address2': '', 'a...",17057333500,+1 705-733-3500,499.259521


#### """CLEANING DATA"""

In [21]:
high_empty_rate(df_yelp)

,columnName,is-nul-check,40%-or higher-empty-data
0,id,0,False
1,alias,0,False
2,name,0,False
3,image_url,0,False
4,is_closed,0,False
5,url,0,False
6,review_count,0,False
7,categories,0,False
8,rating,0,False
9,coordinates,0,False


In [22]:
drop_na(df_yelp)
df_yelp.drop(['alias', 'image_url', 'is_closed', 'url', 'categories', 'coordinates', 'distance', 'phone'], axis=1, inplace=True)

In [23]:
df_yelp_location = df_yelp['location']
df_yelp.drop(['location'], axis=1, inplace=True)

In [24]:
df_yelp['price'] = df_yelp['price'].fillna(df_yelp.price.describe().top)

In [26]:
df_yelp.head(5)

,id,name,review_count,rating,price,display_phone
0,0ALYlK6edo8OnbiwH-b7fA,Lazy Tulip Cafe,59,4.0,$$,+1 705-252-0910
1,FEV75sLYoN1dvSpUCyIePA,Pie Wood Fired Pizza Joint,69,4.0,$$,+1 705-728-7952
2,s-noC8RmKF6FF2TZscl4Ig,The Farmhouse Restaurant,84,3.5,$$,+1 705-737-0522
3,efeu6n1otNmaJPhwNLbLLw,Il Buco Ristorante,24,4.5,$$,+1 705-735-9595
4,ye_x-eM4Cl7zkjxIAHSj3g,Casa Mia Restaurant,23,3.5,$$,+1 705-737-3663


In [28]:
yelp_database = sqlite3.connect('yelp_database.db')
yelp_conn = yelp_database.cursor()

In [29]:
df_yelp.columns

Index(['id', 'name', 'review_count', 'rating', 'price', 'display_phone'], dtype='object')

In [30]:
yelp_conn.execute('''
                    CREATE TABLE IF NOT EXISTS Barrie_locations
                    ([id] VARCHAR,[name] VARCHAR,[review_count] DECIMAL,[rating] DECIMAL,[price] VARCHAR, [display_phone] VARCHAR)
                  ''')

yelp_database.commit()

df_yelp.to_sql('Barrie_locations', yelp_database, if_exists='replace', index=False)

37

In [ ]:
# Get the top 10 restaurants according to their rating
